In [ ]:
# Import necessary Python modules
from sklearn.decomposition import PCA
from mne_icalabel import label_components
from mne.preprocessing import ICA
import copy  # This is a Python module that allows you to copy objects without changing the original object
from scipy import signal
import sklearn as sk
import matplotlib.pyplot as plt
import os
import matplotlib
import mne
import numpy as np
from scipy.stats import zscore
matplotlib.use('Agg')  # disable plotting
mne.viz.set_browser_backend('matplotlib', verbose=None)
mne.set_config('MNE_BROWSER_BACKEND', 'matplotlib')
mne.cuda.init_cuda(verbose=True)

# defining input and output directory
files_in = '../data/in/subjects/'
files_out = '../data/out/subjects/'

EEG_list = []
# loading list of subject names from txt file
names = open("./names.txt", "r")
subject_list = names.read().split('\n')
modes = ['EC', 'EO']
# Read the custom montage
montage_path = r"../data/in/MFPRL_UPDATED_V2.sfp"
montage = mne.channels.read_custom_montage(montage_path)

bad_list = [
    [['AF7'],  # 101 EC
     ['AF7']],  # 101 EO
    [[],  # 102
     ['O2']],  # 102
    [['AF7'],  # 103
     ['AF7']],  # 103
    [[],  # 104
     []],  # 104
    [['AF7'],  # 105
     ['AF7']],  # 105
    [['O1'],  # 111
     ['O1']],  # 111
    [['AFz'],  # 112
     ['AFz']],  # 112
    [['Fp1'],  # 113
     ['Fp1']],  # 113
    [[],  # 114
     []],  # 114
    [[],  # 115
     []],  # 115
    [['Rmastoid'],  # 115
     ['Rmastoid']],  # 115
    [[],  # 117
     []],  # 117
    [['Fp1'],  # 118
     ['Fp1']],  # 118
    [[],  # 119
     []],  # 119
    [[],  # 120
     []],  # 120
    [[],  # 401
     []],  # 401
    [['T7'],  # 402
     ['T7']],  # 402
    [[],  # 403
     []],  # 403
    [[],  # 404
     []],  # 404
    [[],  # 406
     []],  # 406
    [[],  # 407
     []],  # 407
    [[],  # 408
     []],  # 408
    [['T8'],  # 409
     ['T8']],  # 409
    [[],  # 410
     []],  # 410
    [[],  # 411
     []],  # 411
    [[],  # 412
     []],  # 412
    [[],  # 413
     []],  # 413
    [[],  # 414
     []],  # 414
    [[],  # 415
     []],  # 415
    [[],  # 416
     []]  # 416
]

# Define the map of channel names using the provided keys
ch_map = {'Ch1': 'Fp1', 'Ch2': 'Fz', 'Ch3': 'F3', 'Ch4': 'F7', 'Ch5': 'LHEye', 'Ch6': 'FC5',
          # Setting FPz as GND so it matches montage
          'Ch7': 'FC1', 'Ch8': 'C3', 'Ch9': 'T7', 'Ch10': 'GND', 'Ch11': 'CP5', 'Ch12': 'CP1',
          'Ch13': 'Pz', 'Ch14': 'P3', 'Ch15': 'P7', 'Ch16': 'O1', 'Ch17': 'Oz', 'Ch18': 'O2',
          'Ch19': 'P4', 'Ch20': 'P8', 'Ch21': 'Rmastoid', 'Ch22': 'CP6', 'Ch23': 'CP2', 'Ch24': 'Cz',
          'Ch25': 'C4', 'Ch26': 'T8', 'Ch27': 'RHEye', 'Ch28': 'FC6', 'Ch29': 'FC2', 'Ch30': 'F4',
          'Ch31': 'F8', 'Ch32': 'Fp2', 'Ch33': 'AF7', 'Ch34': 'AF3', 'Ch35': 'AFz', 'Ch36': 'F1',
          'Ch37': 'F5', 'Ch38': 'FT7', 'Ch39': 'FC3', 'Ch40': 'FCz', 'Ch41': 'C1', 'Ch42': 'C5',
          'Ch43': 'TP7', 'Ch44': 'CP3', 'Ch45': 'P1', 'Ch46': 'P5', 'Ch47': 'Lneck', 'Ch48': 'PO3',
          'Ch49': 'POz', 'Ch50': 'PO4', 'Ch51': 'Rneck', 'Ch52': 'P6', 'Ch53': 'P2', 'Ch54': 'CPz',
          'Ch55': 'CP4', 'Ch56': 'TP8', 'Ch57': 'C6', 'Ch58': 'C2', 'Ch59': 'FC4', 'Ch60': 'FT8',
          'Ch61': 'F6', 'Ch62': 'F2', 'Ch63': 'AF4', 'Ch64': 'RVEye'}

for i_s, subject in enumerate(subject_list):
    for j_m, mode in enumerate(modes):

        print(subject, mode)

        # defining paths for current subject
        input_path = files_in+subject + '/' + mode + '/'
        output_path = files_out + subject + '/' + mode + '/'

        log_file = output_path+'log2.txt'
        log = open(log_file, "w")

        # loading in files savef from 1filter.py
        EEG = mne.io.read_raw_fif(
            output_path + subject + '_maprenamed&nfiltered.fif', preload=True)

        # MARKING BAD CHANNELS

        # TODO - get bad channels from .txt files here

        bad_txt = bad_list[i_s][j_m]  # get list of bad channels

        picks = mne.pick_channels(EEG.ch_names, include=[], exclude=[])
        original_bads = copy.deepcopy(EEG.info["bads"])

        # This can be used to plot the data with the bad channels marked.
        # Uncomment the two lines of code below to see the plot
        # Replace 'regexp=" ."' with the tentative bad channels

        if bad_txt == [''] or len(bad_txt) == 0:
            pass
        else:
            picks = mne.pick_channels(
                EEG.ch_names, include=[], exclude=bad_txt)
            plot_obj = EEG.plot(order=picks, n_channels=len(picks))
            
        # Change list of bad channels
            original_bads = copy.deepcopy(EEG.info["bads"])
            for bad in bad_txt:
                EEG.info["bads"].append(bad)  # add a single channel
                # add a single channel to the original_bads list
                original_bads.append(bad)
            # EEG_csd.info["bads"].extend(["EEG 051", "EEG 052"])  # add a list of channels
            # original_bads["bads"].extend(["EEG 051", "EEG 052"])  # add a list of channels

            # Print the bad channels to double check
            log.write('Writing Bad Channels to Double check \n')
            log.write(str(EEG.info['bads']) + '\n')
            log.write(str(original_bads) + '\n \n \n')

            # Save the data with the bad channels marked
            # Replace with your desired output directory
            bad_channel_file = output_path + subject + '_badchannels.fif'
            EEG.save(bad_channel_file, overwrite=True)

        # ICA (Independent Component Analysis)

        # Keep a reference to the original, uncropped data
        original_EEG = EEG

        # Crop a copy of the data to three seconds for easier plotting
        cropped_EEG = EEG.copy().crop(tmin=0, tmax=3).load_data()

        # Fit average re-reference to the data
        original_EEG.set_eeg_reference('average')

        # Drop channels #10 and #21 (mastoids) before ICA
        original_EEG.drop_channels(['Rmastoid'])

        # Determine the number of PCA components
        data = original_EEG.get_data().T
        pca = PCA()
        pca.fit(data)

        # Plot the explained variance ratio
        plt.figure(figsize=(10, 5))
        plt.plot(np.cumsum(pca.explained_variance_ratio_))
        plt.xlabel('Number of Components')
        plt.ylabel('Cumulative Explained Variance')
        plt.title('Explained Variance by PCA Components')
        plt.grid(True)
        

        # Define ICA parameters
        n_components = 0.99  # Choose number of ICA components based on PCA
        ica = ICA(
            n_components=n_components,
            max_iter="auto",
            method="infomax",
            random_state=97,
            fit_params=dict(extended=True),
        )

        # Pick only EEG channels
        picks_eeg = mne.pick_types(original_EEG.info, meg=False,
                                   eeg=True, eog=False, stim=False, emg=False, exclude='bads')

        # Fit ICA using only EEG channels
        ica.fit(original_EEG, picks=picks_eeg, decim=3)

        # Plot the ICA components as time series
        ica_ts_plot = ica.plot_sources(
            original_EEG, show_scrollbars=False, show=True)
        # saving the timeseries plot
        

        # Plot the ICA components as topographies in multiple windows
        log.write("Plotting the ICA components as topographies... \n")
        n_components_actual = ica.n_components_

        # Selecting ICA components automatically using ICLabel
        ic_labels = label_components(
            original_EEG, ica, method='iclabel')
        component_labels = ic_labels["labels"]  # Extract the labels
        # Extract the probabilities
        component_probabilities = ic_labels["y_pred_proba"]


        # ICLabel scores
        EEG_list.append(original_EEG)
        print("Initializing labels file \n")
        label_file = output_path+'labels.txt'
        labelf = open(label_file, "r")
        prob_file = output_path+'probs.txt'
        probf = open(prob_file, "r")

        labels = labelf.read().split('\n')
        

        # Exclude components based on label and probability
        exclude_idx = [idx for idx, (label, prob) in enumerate(zip(labels, component_probabilities))
               if label.strip() not in ["brain", "other"] or prob < 0.70]
        
        # Copy the original interpolated EEG data
        reconst_EEG = original_EEG.copy()

        # Apply the ICA transformation, excluding certain components
        ica.apply(reconst_EEG, exclude=exclude_idx)

        # Plot the original data and set the window title
        fig = original_EEG.plot(show_scrollbars=False)

        # Plot the reconstructed data and set the window title
        fig = reconst_EEG.plot(show_scrollbars=False)

        # Save the preprocessed data
        output_dir = r'../data/out'
        preprocessed_file = output_path + subject+'_ICA.fif'
        reconst_EEG.save(preprocessed_file, overwrite=True)

        #################################################################################

        # Interpolate bad channels
        
        # Replace NaN or inf values in channel locations with zero
        new_chs = original_EEG.info['chs'].copy()
        for ch in new_chs:
            ch['loc'] = np.nan_to_num(ch['loc'], nan=0.0, posinf=0.0, neginf=0.0)

        new_info = mne.create_info(
            [ch['ch_name'] for ch in new_chs], original_EEG.info['sfreq'], ch_types='eeg')
        original_EEG = mne.io.RawArray(original_EEG.get_data(), new_info)
        original_EEG.set_montage(mne.channels.make_dig_montage(
            ch_pos={ch['ch_name']: ch['loc'][:3] for ch in new_chs}))
        # Set the bad channels back to the original list
        original_EEG.info['bads'] = original_bads

        # Repeat for cropped_EEG
        new_chs = cropped_EEG.info['chs'].copy()
        for ch in new_chs:
            ch['loc'] = np.nan_to_num(ch['loc'], nan=0.0, posinf=0.0, neginf=0.0)

        new_info = mne.create_info(
            [ch['ch_name'] for ch in new_chs], cropped_EEG.info['sfreq'], ch_types='eeg')
        cropped_EEG = mne.io.RawArray(cropped_EEG.get_data(), new_info)
        cropped_EEG.set_montage(mne.channels.make_dig_montage(
            ch_pos={ch['ch_name']: ch['loc'][:3] for ch in new_chs}))
        # Set the bad channels back to the original list
        cropped_EEG.info['bads'] = original_bads

        # Pick types and interpolate bads
        original_EEG_data = original_EEG.copy().pick_types(
            meg=False, eeg=True, exclude=[])
        original_EEG_data_interp = original_EEG_data.copy().interpolate_bads(reset_bads=False)

        cropped_EEG_data = cropped_EEG.copy().pick_types(meg=False, eeg=True, exclude=[])
        cropped_EEG_data_interp = cropped_EEG_data.copy().interpolate_bads(reset_bads=False)

        # Plot the data before and after interpolation
        for title, data in zip(["cropped orig.", "cropped interp."], [cropped_EEG_data, cropped_EEG_data_interp]):
            with mne.viz.use_browser_backend("matplotlib"):
                fig = data.plot(butterfly=True, color="#00000022", bad_color="r")
            fig.subplots_adjust(top=0.9)
            fig.suptitle(title, size="xx-large", weight="bold")

        # Save the interpolated data
        output_dir = r'../data/out'
        preprocessed_file = output_path+ subject + '_interpolated.fif'
        original_EEG_data_interp.save(preprocessed_file, overwrite=True)

        #################################################################################

        # EPOCHING

        # Define epoch parameters
        name = subject + '_eventchan'  # --> change for each condition
        # Latency rate/Sampling rate
        epoch_no = np.floor(reconst_EEG.get_data(
        ).shape[1] / reconst_EEG.info['sfreq'])

        # Create a list of onset times for your events
        onsets = np.arange(0, reconst_EEG.get_data(
        ).shape[1] / reconst_EEG.info['sfreq'], 1)

        # Create a list of event durations (all zeros if the events are instantaneous)
        durations = np.zeros_like(onsets)

        # Create a list of event descriptions
        descriptions = ['Event'] * len(onsets)

        # Create an Annotations object
        annotations = mne.Annotations(onsets, durations, descriptions)

        # Add the annotations to the Raw object
        reconst_EEG.set_annotations(annotations)

        # Now you can extract the events from the annotations
        events, event_id = mne.events_from_annotations(reconst_EEG)

        # Define epoching parameters
        name = subject + '_epoch'  # --> change the name of condition
        codes = ['1']
        tmin = -0.5  # Start of the epoch (in seconds)
        tmax = 0.5  # End of the epoch (in seconds)

        # Create epochs without rejection to keep all data
        epochs_all = mne.Epochs(reconst_EEG, events, event_id=event_id,
                                tmin=tmin, tmax=tmax, proj=True, baseline=None, preload=True)

        # Apply z-score normalization and keep track of which epochs exceed the threshold
        zscore_threshold = 6
        to_drop = []

        temp_data = np.zeros_like(epochs_all._data)

        for i in range(len(epochs_all)):
            temp_data[i] = zscore(epochs_all._data[i], axis=1)
            if np.any(np.abs(temp_data[i]) > zscore_threshold):
                to_drop.append(i)

        # Now we can drop the epochs that exceeded the threshold
        epochs_all.drop(to_drop)

        # Resample and decimate the epochs
        current_sfreq = epochs_all.info['sfreq']
        # Hz chaging this according to  https://doi.org/10.1046/j.1440-1819.2000.00729.x
        desired_sfreq = 512

        # Apply the resampling
        epochs_all.resample(desired_sfreq, npad='auto')

        # Get the data from all epochs
        data_all = epochs_all.get_data()

        # Plot the data
        epochs_all.plot()

        # Save the filtered data
        # Replace with your desired output directory
        output_dir = r'../data/out'
        preprocessed_file = output_path + subject + '_epoched.fif'
        epochs_all.save(preprocessed_file, overwrite=True)


      

101 EC
Opening raw data file ../data/out/subjects/101/EC/101_maprenamed&nfiltered.fif...
    Range : 0 ... 63449 =      0.000 ...    63.449 secs
Ready.
Reading 0 ... 63449  =      0.000 ...    63.449 secs...


/tmp/ipykernel_3776169/3663576940.py:17: RuntimeWarning: module cupy not found, CUDA not enabled
  mne.cuda.init_cuda(verbose=True)
/tmp/ipykernel_3776169/3663576940.py:120: RuntimeWarning: This filename (../data/out/subjects/101/EC/101_maprenamed&nfiltered.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  EEG = mne.io.read_raw_fif(


Overwriting existing file.
Writing /home/prakhar7/mfprl/sample/neuroconn/batch/../data/out/subjects/101/EC/101_badchannels.fif
Closing /home/prakhar7/mfprl/sample/neuroconn/batch/../data/out/subjects/101/EC/101_badchannels.fif
[done]
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


/tmp/ipykernel_3776169/3663576940.py:160: RuntimeWarning: This filename (/home/prakhar7/mfprl/sample/neuroconn/batch/../data/out/subjects/101/EC/101_badchannels.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  EEG.save(bad_channel_file, overwrite=True)


Fitting ICA to data using 57 channels (please be patient, this may take a while)
Selecting by explained variance: 44 components
Computing Extended Infomax ICA
Fitting ICA took 20.3s.
Creating RawArray with float64 data, n_channels=47, n_times=63450
    Range : 0 ... 63449 =      0.000 ...    63.449 secs
Ready.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

Overwriting existing file.
112 EO
Opening raw data file ../data/out/subjects/112/EO/112_maprenamed&nfiltered.fif...
    Range : 0 ... 49824 =      0.000 ...    99.648 secs
Ready.
Reading 0 ... 49824  =      0.000 ...    99.648 secs...


/tmp/ipykernel_3776169/3663576940.py:378: RuntimeWarning: This filename (../data/out/subjects/112/EC/112_epoched.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_all.save(preprocessed_file, overwrite=True)
/tmp/ipykernel_3776169/3663576940.py:120: RuntimeWarning: This filename (../data/out/subjects/112/EO/112_maprenamed&nfiltered.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  EEG = mne.io.read_raw_fif(


Overwriting existing file.
Writing /home/prakhar7/mfprl/sample/neuroconn/batch/../data/out/subjects/112/EO/112_badchannels.fif
Closing /home/prakhar7/mfprl/sample/neuroconn/batch/../data/out/subjects/112/EO/112_badchannels.fif
[done]
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


/tmp/ipykernel_3776169/3663576940.py:160: RuntimeWarning: This filename (/home/prakhar7/mfprl/sample/neuroconn/batch/../data/out/subjects/112/EO/112_badchannels.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  EEG.save(bad_channel_file, overwrite=True)


Fitting ICA to data using 57 channels (please be patient, this may take a while)
Selecting by explained variance: 27 components
Computing Extended Infomax ICA
Fitting ICA took 10.2s.
Creating RawArray with float64 data, n_channels=30, n_times=49825
    Range : 0 ... 49824 =      0.000 ...    99.648 secs
Ready.
